The goal of this notebook is to develop an end2end text classifier using different embedding like word2vec, fasttext, and doc2vec.

The first pipe is to clean the text data (More cleaning for future data.) 

In [1]:
# Data cleaning
# ===========================================================================
import re
def clean_text(text):
    # replace  . and a space with only a space, then amke all words lower case.
    text = text.replace(". "," ").replace(",","").lower()
    # get rid of the . at the end of each line. 
    cleaned_text = re.sub("\.$","",text)
    
    return cleaned_text
 


class text_clean:
    """
    A class to help with cleaning text data. 
    """
    def fit(self, X, y):
        return self
    def transform(self, X):
        assert isinstance(X,pd.Series), "The input data should be pandas Series."
        X = X.apply(clean_text)
        
        return X


# Word embedding training 
# ===========================================================================
from tqdm import tqdm
import numpy as np
from gensim.models import Word2Vec
from gensim.models import FastText
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

from sklearn.externals import joblib



def _find_part_pii(text, model, sep = " "):
    tokenized_text = text.split(sep)
    
    part_pii = model.wv.doesnt_match(tokenized_text)
    
    return part_pii    



def _extracted_pii2matrix(pii_list, model):
    # set the matrix dimensions
    column_num = model.trainables.layer1_size
    row_num = len(pii_list)
    # initialized the matrix
    pii2vec_mat = np.zeros((row_num, column_num))
    # iterate through the pii_list and assign the vectors to matrix.
    for index, ith_pii in enumerate(tqdm(pii_list)):
        pii2vec_mat[index,:] = model.wv[ith_pii]
    
    return pii2vec_mat



class word_embedding:
    """
    A class to convert words/docs to vectors by applying any model supported by gensim.  
    
    This class will allow continued training on the pre-trained model by assigning
    the model to the pre_trained option in class initialization.  
    
    After training the model, it will dump the word2vec model to the path assigned to 
    dump_file option.  
    
    
    """
    def __init__(self, algo_name = "word2vec", size = 100, min_count = 1, window = 5, workers =1,\
                 epochs = 5, pre_train = None, dump_file = False, continue_train_pre_train = True,
                 re_train_new_sentences = True):
        
        
        assert algo_name in ["word2vec", 'fasttext', 'doc2vec'], \
        "please enter a model name in ['word2vec', 'fasttext', 'doc2vec']"
        
        self.algo_name = algo_name
        self.epochs = epochs 
        self.pre_train = pre_train
        self.dump_file = dump_file 
        self.re_train_new_sentences = re_train_new_sentences
        self.continue_train_pre_train = continue_train_pre_train
        
        # model options
        self.size = size
        self.min_count = min_count
        self.window = window
        self.workers = workers
        
        
    def _algo_init(self):
        if self.algo_name == "word2vec":
            model = Word2Vec(size = self.size, min_count = self.min_count,
                            window = self.window, workers = self.workers)
        elif self.algo_name == "fasttext":
            model = FastText(size = self.size, min_count = self.min_count,
                            window = self.window, workers = self.workers)
        elif self.algo_name == "doc2vec":
            model = Doc2Vec(vector_size = self.size, min_count = self.min_count,
                            window = self.window, workers = self.workers)
            
        self.model = model
        return self

    def _embedding_training(self, sentences, update = False):
        """
        This helper functions will build the vocabulary, train the model and update the self.model
        with the newly trained model.
        
        if update = True, it will update the vocabulary and the model can continue to train.
        If update = False, the model will rebuild a new vocabulary from scratch using the input data.
        """
        updated_model_with_vocab = self.model

        updated_model_with_vocab.build_vocab(sentences, update = update)
        
        updated_model_with_vocab.train(sentences, total_examples = len(sentences), epochs = self.epochs)
        
        # update the model with the trained one. 
        self.model = updated_model_with_vocab
        
    def _pd_to_gensim_format(self, text):
        
        # special handling for doc2vec model. 
        if self.algo_name == "doc2vec":
            documents = [TaggedDocument(sentence.split(" "), [index])\
                          for index, sentence in enumerate(text)] 
            print("Using index for the tags")    
        else:
            documents = [sentence.split(" ") for sentence in text]
            
            
        return documents
            
        
    def fit(self, X):
        """
        The fit method will get use the pre_trained model if the model is assigned to the pre_train option.
        
        If the pre_train is None, then the model will be trained. 
        
        
        If the pre_train model is not None, then the default is to continue training on the new model. 
        Unless option continue_train_pre_train is specified as False. The False option will just assign 
        the pre_train model to self.model
        """
        gensim_X = self._pd_to_gensim_format(text = X)
        
        if self.pre_train is not None:
            
            # update the pre_trained model with new training data
            if self.continue_train_pre_train:
                self.model = self.pre_train
                self._embedding_training(sentences = gensim_X, update = True)
                print("continue training with the pre_train model.")
                
            # no training the pre_trained model. 
            elif not self.continue_train_pre_train:
                self.model = self.pre_train
                print("No training with pre_train model.")
                
            
            return self
        
        else:
            # initialize the model, split the sentence into tokens and train it. 
            self._algo_init()
            self._embedding_training(sentences = gensim_X)
            print("Building new vocabulary and training the {} model".format(self.algo_name))
        
        
        # dump the model to disk
        if isinstance(self.dump_file,str):
            self.model.save(self.dump_file)
            print("Writing the {} model to disk.".format(self.algo_name))
            
        return self
        
    
    def transform(self, X):
        """
        If re_train_new_sentences is True, which is the default setting, 
        the model will be re-trained on the new sentences. 
        This will create word embedding for words not in the original vocabulary.
        This will increase the model inference time since it invovles model training. 
        
        For using word2vec to predict PII data, it is recommended to update the model with new sentences. 
        For fastttext, it is not necessary since it will infer from the character n-grams. The fasttext training
        is much longer than word2vec. 
        """
        gensim_X = self._pd_to_gensim_format(text = X)
        
        # update the embedding with new sentences or train the model. 
        if self.re_train_new_sentences:
            self._embedding_training(sentences = gensim_X, update = True)
            print("transforming while training {} model with new data.".format(self.algo_name))
            
            
        # extract the PII 
        extracted_pii_list = [_find_part_pii(text = text, model = self.model)\
                    for text in tqdm(X) ]
        
        # convert the extracted pii text into vectors.
        piivec_matrix = _extracted_pii2matrix(pii_list = extracted_pii_list,\
                                          model = self.model)
        return piivec_matrix 
                                          

D:\Data_science\Python3.7\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
import pandas as pd
algo_test_data = pd.read_csv("../data/train_text_with_pii_2019_01_05_02_48_24_796403.csv")

## Word2vec testing

In [3]:
testing_embedding = word_embedding(algo_name = 'word2vec', dump_file = './devs/word2vec.bin')
testing_embedding.fit(algo_test_data['Text']);
test_pii_matrix = testing_embedding.transform(algo_test_data["Text"])
test_pii_matrix.shape

Building new vocabulary and training the word2vec model
Writing the word2vec model to disk.
transforming while training word2vec model with new data.


100%|██████████| 800/800 [00:00<00:00, 160424.71it/s]


(800, 100)

In [4]:
pre_trained1 = Word2Vec.load("./word2vec/word2vec_cleaned_300_.bin")
pre_trained2 = Word2Vec.load("./devs/word2vec.bin")
testing_pre_trained = word_embedding(algo_name = "word2vec", pre_train = pre_trained2)
testing_pre_trained.fit(algo_test_data["Text"])
test_pii_matrix = testing_pre_trained.transform(algo_test_data["Text"])
test_pii_matrix.shape

continue training with the pre_train model.
transforming while training word2vec model with new data.


100%|██████████| 800/800 [00:00<00:00, 89112.53it/s]


(800, 100)

## Fasttext testing

In [5]:
testing_embedding = word_embedding(algo_name = 'fasttext', dump_file = "./devs/fasttext.bin")

testing_embedding.fit(algo_test_data["Text"]);
test_pii_matrix = testing_embedding.transform(algo_test_data["Text"])
test_pii_matrix.shape

Building new vocabulary and training the fasttext model
Writing the fasttext model to disk.
transforming while training fasttext model with new data.


100%|██████████| 800/800 [00:00<00:00, 89124.36it/s]


(800, 100)

In [6]:
pre_trained = FastText.load("./devs/fasttext.bin")
testing_pre_trained = word_embedding(algo_name = "fasttext", pre_train = pre_trained)
testing_pre_trained.fit(algo_test_data["Text"])
test_pii_matrix = testing_pre_trained.transform(algo_test_data["Text"])
test_pii_matrix.shape

continue training with the pre_train model.
transforming while training fasttext model with new data.


100%|██████████| 800/800 [00:00<00:00, 100336.20it/s]


(800, 100)

## doc2vec testing

In [7]:
testing_embedding_doc2vec = word_embedding(algo_name = 'doc2vec', dump_file = "./devs/doc2vec.bin")
testing_embedding_doc2vec.fit(X = algo_test_data['Text'])

test__doc2vec_pii_matrix = testing_embedding_doc2vec.transform(algo_test_data['Text'])
test__doc2vec_pii_matrix.shape

Using index for the tags
Building new vocabulary and training the doc2vec model
Writing the doc2vec model to disk.
Using index for the tags
transforming while training doc2vec model with new data.


100%|██████████| 800/800 [00:00<00:00, 200384.78it/s]


(800, 100)

In [8]:
pre_trained = FastText.load("./devs/doc2vec.bin")
testing_pre_trained = word_embedding(algo_name = "doc2vec", pre_train = pre_trained)
testing_pre_trained.fit(algo_test_data["Text"])
test_pii_matrix = testing_pre_trained.transform(algo_test_data["Text"])
test_pii_matrix.shape

Using index for the tags
continue training with the pre_train model.
Using index for the tags
transforming while training doc2vec model with new data.


100%|██████████| 800/800 [00:00<00:00, 201358.81it/s]


(800, 100)